In [8]:
# Importanto bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm



In [11]:
# Importando os dados das inscrições 
df = pd.read_csv('data/Inscritos(Total).csv', sep=';')

# 4 edições para treino
subs_training = df.loc[df['Edição'] != '2018.1'] 

# Última edição para teste
subs_test = df.loc[df['Edição'] = '2018.1'] 


In [14]:

X = subs_t.iloc[:, :-1].values  # independent variables
Y = subs_total.iloc[:, -1:].values


y "Selecionado"


SyntaxError: invalid syntax (<ipython-input-14-100a91691fab>, line 1)